# Introduction to Database Design
---------------------------------------------------------------------------------------------------------

In our previous session on databases, we introduced some of the fundamental concepts and definitions applicable to databases in general, along with a brief intro to SQL and SQLite in particular. Some use cases and platforms were also discussed.

In this session, we are going to dig a little deeper into databases as representions of systems and processes. A database with a single table may not feel or function much differently from a spreadsheet. Much of the benefit of using databases results from designing them as models of complex systems in ways that spreadsheets just can't do:

* Inventory control and billing
* Human resources
* Blogs
* Ecosystems

For this session we are also going to play as we go, so let's begin by installing the iPython SQL library developed by Catherine Devlin and others at [https://github.com/catherinedevlin/ipython-sql](https://github.com/catherinedevlin/ipython-sql).

Note that in order to run SQL commands within a Jupyter Notebooks, code blocks need to begin with a 'magic' function:

%sql
for inline SQL or

%%sql
for multiple lines of SQL in a code block.

## Install and load the library.

In [7]:
#!pip3 install ipython-sql # https://github.com/catherinedevlin/ipython-sql
#!pip install ipython-sql # https://github.com/catherinedevlin/ipython-sql

In [8]:
%load_ext sql
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @None'